In [1]:
import os
import pandas as pd

In [2]:
def load_fmap_log(folder_path):
    csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                 if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.csv')]
    aggregate_result = pd.DataFrame()
    for record in csv_files:
        df = pd.read_csv(record)
        aggregate_result = aggregate_result.append(df.groupby(['model', 'rank', 'beta', 'lambda_l2', 'lambda_keyphrase', 'lambda_latent', 'lambda_rating', 'learning_rate', 'epoch', 'corruption', 'optimizer'], as_index=False)
 .agg({'F-MAP@5':'mean', 'F-MAP@10':'mean','F-MAP@20':'mean'})).sort_values(by=['beta'])
    return aggregate_result

In [3]:
df = load_fmap_log(folder_path='tables/beer_fmap/')

In [4]:
df

,model,rank,beta,lambda_l2,lambda_keyphrase,lambda_latent,lambda_rating,learning_rate,epoch,corruption,optimizer,F-MAP@5,F-MAP@10,F-MAP@20
0,E-CDE-VAE,100,0.0000,0.001,1,1,1,0.0005,600,0.4,Adam,0.009953,0.006610,0.007389
0,E-CDE-VAE,100,0.0000,0.001,1,1,1,0.0005,600,0.4,Adam,0.035087,0.037047,0.036432
0,E-CDE-VAE,100,0.0000,0.001,1,1,1,0.0005,600,0.4,Adam,0.026547,0.030944,0.031129
1,E-CDE-VAE,100,0.0001,0.001,1,1,1,0.0005,600,0.4,Adam,0.029780,0.031860,0.032814
1,E-CDE-VAE,100,0.0001,0.001,1,1,1,0.0005,600,0.4,Adam,0.029207,0.029273,0.026059
1,E-CDE-VAE,100,0.0001,0.001,1,1,1,0.0005,600,0.4,Adam,0.032520,0.029793,0.032032
2,E-CDE-VAE,100,0.0010,0.001,1,1,1,0.0005,600,0.4,Adam,0.000000,0.000000,0.000000
2,E-CDE-VAE,100,0.0010,0.001,1,1,1,0.0005,600,0.4,Adam,0.000000,0.000000,0.000000
2,E-CDE-VAE,100,0.0010,0.001,1,1,1,0.0005,600,0.4,Adam,0.000000,0.000000,0.000000
3,E-CDE-VAE,100,0.0100,0.001,1,1,1,0.0005,600,0.4,Adam,0.000000,0.000000,0.000000


In [8]:
(df.loc[(df['F-MAP@5'] != 0) & (df['F-MAP@10'] != 0) & (df['F-MAP@20'] != 0)].groupby(['model', 'rank', 'beta', 'lambda_l2', 'lambda_keyphrase', 'lambda_latent', 'lambda_rating', 'learning_rate', 'epoch', 'corruption', 'optimizer'], as_index=False)
.agg({'F-MAP@5':'mean', 'F-MAP@10':'mean','F-MAP@20':'mean'})).sort_values(by=['beta'])

,model,rank,beta,lambda_l2,lambda_keyphrase,lambda_latent,lambda_rating,learning_rate,epoch,corruption,optimizer,F-MAP@5,F-MAP@10,F-MAP@20
0,E-CDE-VAE,100,0.0000,0.001,1,1,1,0.0005,600,0.4,Adam,0.023862,0.024867,0.024983
1,E-CDE-VAE,100,0.0001,0.001,1,1,1,0.0005,600,0.4,Adam,0.030502,0.030309,0.030301
2,E-CDE-VAE,100,0.0100,0.001,1,1,1,0.0005,600,0.4,Adam,0.024520,0.017971,0.014078
3,E-CDE-VAE,100,0.1000,0.001,1,1,1,0.0005,600,0.4,Adam,0.000750,-0.000307,-0.000073
4,E-CDE-VAE,100,1.0000,0.001,1,1,1,0.0005,600,0.4,Adam,0.001182,0.000073,-0.002565


# Bar Plot for best beta

In [ ]:
def pandas_bar_plot(df, x, y, hue, x_name, y_name, folder='figures', name='unknown', save=True):
    fig, ax = plt.subplots(figsize=(6, 3))
    sns.barplot(ax=ax, x=x, y=y, hue=hue, data=df, errwidth=1, edgecolor='black', facecolor=(1, 1, 1, 0)) #, errwidth=0.5

    num_category = len(df[x].unique())
    hatch = None
    hatches = itertools.cycle(['//', '**', '////', '----', 'xxxx', '\\\\\\\\', ' ', '\\', '...', 'OOO', "++++++++"])
    for i, bar in enumerate(ax.patches):
        if i % num_category == 0:
            hatch = next(hatches)
        bar.set_hatch(hatch)

    plt.xlabel(x_name)
    plt.ylabel(y_name)
    #plt.xticks(rotation=15)
    plt.legend(loc='upper left', ncol=5)
    # if 'Precision' not in y:
    ax.legend_.remove()
    plt.tight_layout()
    if save:
        fig_path = load_yaml('config/global.yml', key='path')['figs']
        plt.savefig("{2}/{0}/{1}_bar.pdf".format(folder, name, fig_path), format="pdf")
        plt.savefig("{2}/{0}/{1}_bar.png".format(folder, name, fig_path), format="png")

        fig_leg = plt.figure(figsize=(12, 0.7))
        ax_leg = fig_leg.add_subplot(111)
        ax_leg.legend(*ax.get_legend_handles_labels(), loc='center', ncol=10)
        ax_leg.axis('off')
        fig_leg.savefig('figs/bar_legend.pdf', format='pdf')

    else:
        plt.show()
    plt.close()

# Line Plot for different beta